In [1]:
from pyspark import SparkContext
import numpy as np
import geopy.distance
sc = SparkContext.getOrCreate()


KeyboardInterrupt: 

In [2]:
def splitAndTake(line):
    arr = line.split('\t')
    country = str(arr[1])
    capital = str(arr[2])
    lat = float(arr[3].strip().replace(",","."))
    lon = float(arr[4].strip().replace(",","."))
    return(country, capital, lat, lon)

In [3]:
capital_rdd = sc.textFile("datasets/Capitals.txt")
capital_rdd = capital_rdd.map(lambda line: splitAndTake(line))
capital_rdd = capital_rdd.filter(lambda x: x[2] >= -90)
capital_rdd = capital_rdd.filter(lambda x: x[2] <= 90)
capital_rdd = capital_rdd.sortBy(lambda x: x[2],ascending=True) # dropping some capitals due to latitude problems
#capital_rdd.take(10)

In [4]:
def findDistanceBetweenCapitals(line):
    f_coord = (line[0][2], line[0][3])
    s_coord = (line[1][2], line[1][3])
    f_capital = line[0][1]
    s_capital = line[1][1]
    dist = geopy.distance.geodesic(f_coord,s_coord).km
    return (f_capital, s_capital, dist)



In [5]:
country_pairs = capital_rdd.cartesian(capital_rdd)
country_pairs = country_pairs.filter(lambda x: x[0] != x[1])
capitals = country_pairs.map(lambda line: findDistanceBetweenCapitals(line))
capitals = capitals.sortBy(lambda line: line[2], ascending=True)
capitals.take(10)

[('Jerusalem ', 'East Jerusalem ', 0.0),
 ('East Jerusalem ', 'Jerusalem ', 0.0),
 ('Rome ', 'Vatican ', 2.72613532590367),
 ('Vatican ', 'Rome ', 2.72613532590367),
 ('Kinshasa ', 'Brazzaville ', 6.454518401229527),
 ('Brazzaville ', 'Kinshasa ', 6.454518401229527),
 ('Marigot ', 'The Valley ', 16.73665420507278),
 ('The Valley ', 'Marigot ', 16.73665420507278),
 ('Gustavia ', 'Marigot ', 30.78062900685881),
 ('Marigot ', 'Gustavia ', 30.78062900685881)]